# Armbot disinfection

In [1]:
import pickle
import trimesh as tm
import pyrender
import numpy as np
import visibility
from importlib import reload
reload(visibility)
from visibility.visibility import Visibility
from joblib import Parallel,delayed
from tqdm import tqdm
from scipy.sparse import lil_matrix
from klampt import WorldModel,Geometry3D
from klampt import vis
from klampt.math import so3,se3
import trimesh
import os
import random
import numpy as np
import pdb
import math
from klampt.model.create import moving_base_robot
from klampt.io import resource
from klampt.math import vectorops,so3
from klampt.model.collide import WorldCollider
from klampt.model import ik
import time
import numpy as np
from tqdm import tqdm
from klampt import IKObjective
import networkx as nx
from klampt.model.trajectory import RobotTrajectory,Trajectory
from klampt.model.create import *
from klampt import vis
from klampt.robotsim import Geometry3D,WorldModel
from klampt.math import se3,so3,vectorops
from klampt.vis import colorize
import sys
import time
from klampt.model import trajectory
from threading import Thread,Lock,Event



configs_file = 'configs.p'
robot_file = "./data/armbot.rob"
movie_dir = '../media/movies/armbot/30_min/{:06d}.png'

results_dir = './3D_results'
mesh_file = './remeshed_hospital_room_full_35k.obj'

def generate_video(mesh_file = './remeshed_hospital_room_full_35k.obj',
                   robot_file = "./data/armbot.rob",
                   movie_dir = './media/movies/armbot/30_min',
                   tour_file = 'armbot_currTSP.txt',
                   results_dir = './3D_results',
                   experiment = '30_min',
                   resolution = 250,
                   skip = 1,
                   dt = 0.4,
                   max_step = 20,
                   scale = 4,
                   linknum = 11):

    mesh_name = mesh_file.split('.')[-2]
    results_dir = results_dir+'/{}'.format(mesh_name)
    if(not os.path.exists(results_dir)):
        os.mkdir(results_dir)

    results_dir = results_dir + '/{}'.format(experiment)
    if(not os.path.exists(results_dir)):
        os.mkdir(results_dir)

    results_dir = results_dir + '/{}'.format('armbot')
    if(not os.path.exists(results_dir)):
        os.mkdir(results_dir)


    pcd_file = results_dir + "/armbot_used_points_{}_divs.pcd".format(resolution)
    reachable_points_file = results_dir + '/armbot_reachable_{}_divs.p'.format(resolution)
    irradiance_file = results_dir + '/armbot_irradiance_matrix_{}_divs.p'.format(resolution)
    sampling_places_file = results_dir + '/armbot_sampling_places_{}_divs.p'.format(resolution)
    configs_file = results_dir + '/armbot_configs_{}_divs.p'.format(resolution)
    solutions_file = results_dir + '/armbot_solutions_{}_divs.p'.format(resolution)
    adjacency_file = results_dir + '/armbot_adjacency_{}_divs.p'.format(resolution)
    roadmap_file = results_dir + '/armbot_roadmap_{}_divs.p'.format(resolution)
    node_coords_file = results_dir + '/armbot_node_coords_{}_divs.p'.format(resolution)

    irradiance_matrix = pickle.load(open(irradiance_file,'rb'))
    reachable = pickle.load(open(reachable_points_file,'rb'))
    configs = pickle.load(open(configs_file,'rb'))
    sampling_places = pickle.load(open(sampling_places_file,'rb'))

    #creating movie directory
    os.makedirs(movie_dir, exist_ok=True)
    # fixing movie paths
    movie_folder = movie_dir
    movie_dir += '/{:06d}.png'
    
    sampling_places = sampling_places[reachable]
    configs = np.array(configs)[reachable].tolist()
    mu_single = 60*60*80*0.5*irradiance_matrix
    print("\n\n number of reachable points : {} \n".format(np.sum(reachable)))




    #process irradiation spots
    irradiance_matrix = pickle.load(open(irradiance_file,'rb'))

    vis_tester =Visibility(mesh_file,res = 512, useShader = True,createWnd = True)
    m = vis_tester.mesh
#     m=Mesh(mesh_file)
#     areas = m.area()
    areas = m.area()
    
    
    areas = m.area()
    visible_points = np.asarray(((irradiance_matrix.sum(axis = 0)) > 0)).flatten()
    visible_areas = areas[visible_points]

    total_visible_areas = visible_areas.sum()
    total_visible_areas


#     irradiance_matrix = irradiance_matrix[reachable]

    visible_points = np.asarray(((irradiance_matrix.sum(axis = 0)) > 0)).flatten()

    visible_irradiance = irradiance_matrix[:,visible_points]


    visible_areas = areas[visible_points]
    # we then define the visible area weights:



    visible_area_weights = np.exp(100*(visible_areas/visible_areas.sum()))
    visible_area_weights[:] = 1


    solutions = pickle.load(open(solutions_file,'rb'))
    visible_points = np.asarray(((irradiance_matrix.sum(axis = 0)) > 0)).flatten()
    visible_irradiance = irradiance_matrix[:,visible_points]



    # we now calculate the euclidean distance matrices for all the non-zero points:
    times = solutions[0]
    total_time = times.sum()
    points_mask = times>0.5
    used_points = sampling_places[points_mask,:]
    print(used_points.shape)
    mus = 80*np.array(np.matmul(visible_irradiance.todense().transpose(),times)).flatten()
    coverage = visible_areas[mus > 280].sum()/243.6
    print('Total Environment Coverage = {:.2f}%'.format(100*coverage))

    adjacency_matrix = pickle.load(open(adjacency_file,'rb'))
    roadmap = pickle.load(open(roadmap_file,'rb'))
    node_coords = pickle.load(open(node_coords_file,'rb'))


    distances = np.zeros(shape = (adjacency_matrix.shape[0]+1,adjacency_matrix.shape[0]+1))
    distances[1:,1:] = 100*adjacency_matrix.copy()
    from getToursAndPaths import getTour, readTourFile, getPathFromPrm, getFinalPath

#     getTour(distances, 'currTSP') # We just have an arbitrary name since it doesn't matter - can change this so that user can input filename if desired
    tour = readTourFile(tour_file, used_points)
    tour = (np.array(tour[1:])-1).tolist()
    indices = np.array(range(sampling_places.shape[0]))
    used_indices = indices[points_mask]
    tour_indices = used_indices[tour]

    # We then calculate the total distance travelled - and compute final trajectory:
    total_distance = 0 
    final_trajectory = []
    for i in range(len(tour)-1):
        current_point = tour[i]
        next_point = tour[i+1]
        total_distance += adjacency_matrix[current_point,next_point]
        nodes_path = nx.algorithms.shortest_path(roadmap,source = current_point,target = next_point,weight = 'weight')
    #     print(nodes_path)
    #     print(nodes_path)
        traj = [node_coords[j][:12] for j in nodes_path]
    #     print(trajectory)
        final_trajectory.append(traj[:-1])
    #     final_trajectory.append(pathDict[current_point,next_point])
    print("Total Distance Travelled by the EE = {} | Total Time Spent in Transit = {} ".format(total_distance,total_distance/30))






    #prepping animations
    time_allowances = np.linspace(0.00,0.5,50)
    times = solutions[0]
    partial_times  = []
    for allowance in time_allowances:
        partial_time = np.zeros(sampling_places.shape[0])

        for k in range(len(tour_indices)-1):
            total_time = times[tour_indices[:k+1]].sum()/(3600)
            if(total_time >= allowance):
                k = k-1
                break
        partial_time[tour_indices[:k+1]] = times[tour_indices[:k+1]]
        print(partial_time.sum()/3600)
        partial_times.append(partial_time)

    # trajectory_times:

    total_times = []
    for k in range(len(tour_indices)):
        partial_time = np.zeros(sampling_places.shape[0])
        partial_time[tour_indices[:k+1]] = times[tour_indices[:k+1]]
        total_times.append(partial_time.sum()/3600)
    #     print(total_times[-1])
        partial_times.append(partial_time)
    total_times = np.array(total_times)

    # we then determine the strategic stops
    first = np.argmin(np.abs(total_times-0.125))
    second = np.argmin(np.abs(total_times-0.25))
    third =  np.argmin(np.abs(total_times-0.375))
    fourth = total_times.shape[0] - 1



    stop_spots = [first,second,third,fourth]
    timed_trajectory = []
    trajectory_times = []
    prev_time = 0
    for index,path in enumerate(final_trajectory):
        section_length = len(path)
        section_times = np.zeros(section_length) + prev_time
        section_times = section_times + np.arange(section_length)*((total_times[index]-prev_time)/section_length)
        section_times = section_times*3600
        prev_time = total_times[index]
        timed_trajectory.extend(path)
        trajectory_times.extend(section_times)

    # setting up basic animations


    vis.init('PyQt')
    # del robot
    a = Geometry3D()
    res = a.loadFile(mesh_file)

    b = Geometry3D()
    res = b.loadFile(pcd_file)
    if(not res):
        print('failed loading geometry!')
    w = WorldModel()

    res = w.loadElement(robot_file)
    print(res)
#     w,robot,lamp,collider = setup_robot_and_light(mesh_file = mesh_file)
    robot = w.robot(0)
    cfig = robot.getConfig()
    cfig[2] = 0.08
    robot.setConfig(cfig)
    w.makeRigidObject("a")
    w.rigidObject(0).geometry().set(a)
    w.makeRigidObject("b")
    w.rigidObject(1).geometry().set(b)

    robotTraj = RobotTrajectory(robot,milestones = timed_trajectory,times = trajectory_times).discretize(dt)
    linkTraj = robotTraj.getLinkTrajectory(linknum,dt) #.getPositionTrajectory([0,0,0])

    times = solutions[0]
    final_times = []
    for this_time in robotTraj.times:
        for j in range(len(tour_indices)):
            partial_time = np.zeros(sampling_places.shape[0])
            partial_time[tour_indices[:j+1]] = times[tour_indices[:j+1]]
            total_time = partial_time.sum()
    #         print(this_time,total_time)
            if(total_time >= this_time):
                break
        partial_time[tour_indices[j]] = this_time-times[tour_indices[:j]].sum()
        final_times.append(partial_time)
    final_times = np.array(final_times)
    # robot,w
    value = None
    cmap = None
    feature = None
#     time.sleep(20*np.random.rand())
    vis.add("A",a)
    # vis.add("B",b,pointSize = 15)
    vis.add("robot",robot)
    curr_milestones = sampling_places[tour_indices[:1],:]
    full_trajectory = []
    for i in final_trajectory:
        full_trajectory.extend(i)
    traj = trajectory.Trajectory(milestones= full_trajectory)#curr_milestones.tolist())
    # vis.add("traj",traj)
    # we then calculate the percentage of disinfection fluence:
    # disinfected_vecs = []
    partial_times_array = np.array(partial_times).transpose()
    # for times in partial_times:
    # mu = 80*np.matmul(visible_irradiance.transpose().toarray(),final_ties)
    # disinfected_vecs = np.clip(mu/(1.1*280),0,1).transpose()

    # we then add all zeros to begin the illustration
    # tmp = [np.zeros(disinfected_vecs[0].shape)]
    # tmp[0][0] = 1
    # disinfected_vecs_new = disinfected_vecs.copy()
    vis.visualization.setBackgroundColor(1, 1, 1, a=1)

    vis.show()
    robot.link(11).appearance().setColor(210/255,128/255,240/255,1)

    # eliminating draw distance
    vis.lock()
    # time.sleep(0.5)
    vp = vis.getViewport()
    # vp.h = 640
    # vp.w = 640
    vp.clippingplanes = [0.1,10000]
    tform = pickle.load(open('tform.p','rb'))
    vp.setTransform(tform)
    vp.w = 1853//scale
    vp.h = 1123//scale
    # vis.setViewport(vp)
    vis.scene().setViewport(vp)
    vis.unlock()
    time.sleep(1)
    new_partial_times = final_times
    max_t = 0
    for i in partial_times:
        max_t = max(max_t,np.max(i))

        
        #animation steps
    tholds = [0.25,0.5,0.75,0.999995]
    thold_index = 0
    thold = tholds[thold_index]
    # i = 56
    # while vis.shown():
    done = get_done_set(movie_folder)
    total_frames = set(range(len(robotTraj.milestones)))
    print(len(total_frames),len(done))
    to_do = total_frames - done
    print('todo = : {}'.format(to_do))
#     pdb.set_trace()
    sc = Screenshotter(movie_dir)
    time.sleep(5)
    for i in sorted(list(to_do))[skip::max_step]:
    #     time.sleep((1/120)*solutions[0][tour_indices[i]])
    #     i = i%(len(disinfected_vecs_new))
        times = new_partial_times[i]
        disinfection_fractions = areas.copy()
        disinfection_fractions[:] = 0
        mu = 80*np.matmul(visible_irradiance.transpose().toarray(),times.transpose())
        disinfected_vecs = np.clip(mu/(1.1*280),0,1).transpose()
        if(i == 0 ):
            disinfected_vecs[0] = 100
#         print(disinfected_vecs.sum())
        #     disinfected_vecs[0] = 0.0001
    #     if(times.sum()/total_time >= thold):
    #         print('\nstopping at {} % due to total disinfection time {}\n'.format(thold,times.sum()/total_time))
    #         thold_index = (thold_index+1)%4
    #         thold = tholds[thold_index]
    #     if(i == 57):
    #         break
    #     time.sleep(0.2)
        fraction = areas[visible_points][disinfected_vecs >= 1].sum()/250.2
        
    #     traj = trajectory.Trajectory(milestones=so_far)

        disinfection_fractions[visible_points] = disinfected_vecs



        start = time.time()
        
        a_app = colorize.colorize(w.rigidObject(0),disinfection_fractions,'RdYlGn','faces',lighting=[0,0,-1])
        end = time.time()
    #     b_app = colorize.colorize(w.rigidObject(1),times,'magma')
    #     b_app.appearance().setPointSize(10)
    #     curr_pos = sampling_places[tour_indices[i],:]
        vis.lock()
        traj = Trajectory(milestones = linkTraj.getPositionTrajectory([0,0,0]).milestones[:i])

#         if(i != 0):
#         vis.remove("A")
        vis.add("A",a_app)
        
        if(i!= 0 ):
            vis.add("traj",traj)
        vis.addText('disinfection_progress','Reachable Coverage = {:.1f} %'.format(100*fraction),position = [10//scale,10//scale],size = 36//scale)
        vis.addText('disinfection_time','Time = {:.2f} minutes'.format(times.sum()/60),position = [10//scale,50//scale],size = 36//scale)
        robot.setConfig(robotTraj.milestones[i])

    #     vis.setAttribute("B","size",10)
    #     vis.remove("B")
    #     vis.add("B",b_app)
    #     vis.dirty("B")
    #     vis.remove("traj")

        vis.unlock()
#         time.sleep(1)
        sc.event.clear()
        sc.i = i
        vis.threadCall(sc.take_screenshot)
        sc.event.wait()
#         vis.threadCall(lambda :vis.scene().program.save_screen(movie_dir.format(i)))
    #     vis.scene().program.save_screen('{}.png'.format(i),multithreaded = False)
        print('\n {} took {}\n '.format(i,end-start))
    #     break
#         time.sleep(0.2)
        

    #     i +=1
    #     time.sleep(5)
    vis.kill()
    
class Screenshotter:
    def __init__(self,movie_dir):
        self.event = Event()
        self.movie_dir = movie_dir
        self.i = 0
        self.event.clear()
    def take_screenshot(self):
        vis.scene().program.save_screen(self.movie_dir.format(self.i))
        self.event.set()
        
def get_done_set(movie_dir):
    from glob import glob
    import pandas as pd
    a = glob(movie_dir + '/*.png')
    if(a):
        a = pd.Series(a)
        b = a.str.split('/',expand = True).iloc[:,-1]
        c = b.str.split('.', expand = True).iloc[:,0].astype(int)
        d = set(c)
#         print(d)
        return d
    else:
        return set({})

In [2]:
from joblib import Parallel,delayed
# generate_video(skip = 1,max_step = 1,dt = 0.5,scale = 1)
max_step = 5
Parallel(n_jobs = max_step,prefer = 'processes')(delayed(generate_video)(skip = skip, max_step = max_step,scale = 1) for skip in range(0,max_step))

[None, None, None, None, None]

In [ ]:
help(vis.init)

In [ ]:
help(vis.threadCall)